Adding Over end runs

In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from xgboost import XGBRegressor
from sklearn.preprocessing import PolynomialFeatures
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [3]:
# df= pd.read_csv('/content/drive/MyDrive/Evarcity/Homework-10days/CricDataset_9thJuly-16Over.csv')
df= pd.read_csv('https://raw.githubusercontent.com/RamanujGitHub/DeepLearning_Repo/main/Sagarnil/CricDataset_9thJuly-16Over.csv?token=GHSAT0AAAAAACCIJSJKVWTXR553B2RGU7T6ZFXNWMQ')

In [4]:
# pca_acc = pd.DataFrame(pca_accuracy_all)
# sns.set(style='darkgrid')
# # Create the line plot using Seaborn
# sns.lineplot(x= 0, y= 1, data='pca_acc')
# # Add labels and title
# plt.xlabel('Number of PCA Components')
# plt.ylabel('Accuracy')
# plt.title('Accuracy vs. Number of PCA Components')
# # Show the plot
# plt.show()

In [5]:
pv= pd.pivot_table(data=df, index = ['match_id','total_scores','league_season','league_name','league_gender',\
                     'is_international','match_name','ground_name','home_team_id','away_team_id','match_year'],\
                   columns  = 'over_num',\
                   values = ['runs_scored','total_wickets','fours_scored','sixes_scored','total_wides','total_no_balls','total_byes','total_leg_byes'],
                   )


pv.columns = pv.columns.map('{0[1]}_Over_{0[0]}'.format)
pv2=pv.reset_index()
y = pv2['total_scores']
pv3 = pv2.drop('total_scores',axis=1)
pv3 = pv3.drop('match_id',axis=1)
obj=pv3[['league_season',	'league_name',	'league_gender',	'match_name',	'ground_name']]
#pv2 and y
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
pvle = pv3.copy()
for x in pvle:
  pvle[x] = le.fit_transform(pvle[x])

scaler = StandardScaler()
X_scaled = scaler.fit_transform(pvle)
Scaled_df=pd.DataFrame(X_scaled)
Scaled_df.columns = pvle.columns
#Perform Dimentionality Reduction
pca = PCA(n_components=35)
X_pca = pca.fit_transform(Scaled_df)
X_pca = pd.DataFrame(X_pca)

In [6]:
Scaled_df.shape

(2851, 137)

In [7]:
pd.DataFrame(pv3).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2851 entries, 0 to 2850
Columns: 137 entries, league_season to 16_Over_total_wides
dtypes: float64(128), int64(4), object(5)
memory usage: 3.0+ MB


In [8]:
models = pd.DataFrame(columns=["Model","MAE","MSE","RMSE","R2 Score","RMSE (Cross-Validation)"])
prediction_all = pd.DataFrame()

In [9]:
from sklearn.model_selection import train_test_split
X = X_pca.copy()
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.3, shuffle= True, random_state = 42)
prediction_all['RealRuns']=y_test

Defining several evaluation functions for convenience

In [10]:

def rmse_cv(model):
    rmse = np.sqrt(-cross_val_score(model, X, y, scoring="neg_mean_squared_error", cv=5)).mean()
    return rmse


def evaluation(y, predictions):
    mae = mean_absolute_error(y, predictions)
    mse = mean_squared_error(y, predictions)
    rmse = np.sqrt(mean_squared_error(y, predictions))
    r_squared = r2_score(y, predictions)
    return mae, mse, rmse, r_squared

Machine Learning Models

Linear Regression

In [11]:
lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)
predictions = lin_reg.predict(X_test)
mae, mse, rmse, r_squared = evaluation(y_test, predictions)
print("MAE:", mae)
print("MSE:", mse)
print("RMSE:", rmse)
print("R2 Score:", r_squared)
print("-"*30)
rmse_cross_val = rmse_cv(lin_reg)
print("RMSE Cross-Validation:", rmse_cross_val)


new_row = {"Model": "LinearRegression","MAE": mae, "MSE": mse, "RMSE": rmse, "R2 Score": r_squared, "RMSE (Cross-Validation)": rmse_cross_val}
models = models.append(new_row, ignore_index=True)
prediction_all['LinearRegression']=predictions

MAE: 10.737364152450583
MSE: 189.45761940737083
RMSE: 13.764360479418244
R2 Score: 0.8397260256681962
------------------------------
RMSE Cross-Validation: 13.965404197851138


<ipython-input-11-09c027909717>:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  models = models.append(new_row, ignore_index=True)


In [12]:
pca_accuracy_all = np.empty((0,2), float)
pcas = [1, 2, 3, 4,5,6,7,8,9,10,11,12,13,14]
for x in pcas:
  pca = PCA(n_components=x)
  X_pca = pca.fit_transform(Scaled_df)
  X_pca = pd.DataFrame(X_pca)
  X = X_pca.copy()
  X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.3, shuffle= True, random_state = 42)
  lin_reg = LinearRegression()
  lin_reg.fit(X_train, y_train)
  predictions = lin_reg.predict(X_test)
  mae, mse, rmse, r_squared = evaluation(y_test, predictions)
  pca_accuracy_all = np.append(pca_accuracy_all,np.array([[x,r_squared*100]]), axis=0)
print(pca_accuracy_all)


[[ 1.          2.17160366]
 [ 2.          6.08525752]
 [ 3.         61.36820949]
 [ 4.         66.20955304]
 [ 5.         72.55797949]
 [ 6.         73.22077208]
 [ 7.         74.82666751]
 [ 8.         76.64623955]
 [ 9.         76.94092126]
 [10.         77.32909331]
 [11.         77.08993188]
 [12.         77.49220075]
 [13.         77.35150979]
 [14.         78.11498618]]


In [13]:
# plt.lineplot(pca_accuracy_all[:, 0], pca_accuracy_all[:, 1], cmap='viridis')
# plt.xlabel('PCA Component')
# plt.ylabel('Acuracy')
# plt.title(f'PCA Components with Accuracy  ')
# plt.colorbar(label='Class')
# plt.show()


* MAE: 11.274825060449345
* MSE: 207.09421289739922
* RMSE: 14.390768321997239
* R2 Score: 0.8248061351873422

* Create a visualization of no. of PCA components with relation to accuracy

Ridge Regression

In [14]:
ridge = Ridge()
ridge.fit(X_train, y_train)
predictions = ridge.predict(X_test)

mae, mse, rmse, r_squared = evaluation(y_test, predictions)
print("MAE:", mae)
print("MSE:", mse)
print("RMSE:", rmse)
print("R2 Score:", r_squared)
print("-"*30)
rmse_cross_val = rmse_cv(ridge)
print("RMSE Cross-Validation:", rmse_cross_val)

new_row = {"Model": "Ridge","MAE": mae, "MSE": mse, "RMSE": rmse, "R2 Score": r_squared, "RMSE (Cross-Validation)": rmse_cross_val}
models = models.append(new_row, ignore_index=True)
prediction_all['Ridge']=predictions

MAE: 12.786526833073259
MSE: 258.6907944839545
RMSE: 16.083867522581578
R2 Score: 0.7811573802907068
------------------------------
RMSE Cross-Validation: 15.75716163203847


<ipython-input-14-1b2467b1c80b>:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  models = models.append(new_row, ignore_index=True)


In [15]:
# # Plot the PCA components with accuracy
# plt.scatter(X_pca[:, 0], X_pca[:, 1], c=y, cmap='viridis')
# plt.xlabel('PCA Component 1')
# plt.ylabel('PCA Component 2')
# plt.title(f'PCA Components with Accuracy: {accuracy:.2f}')
# plt.colorbar(label='Class')
# plt.show()

Lasso Regression

In [16]:
lasso = Lasso()
lasso.fit(X_train, y_train)
predictions = lasso.predict(X_test)

mae, mse, rmse, r_squared = evaluation(y_test, predictions)
print("MAE:", mae)
print("MSE:", mse)
print("RMSE:", rmse)
print("R2 Score:", r_squared)
print("-"*30)
rmse_cross_val = rmse_cv(lasso)
print("RMSE Cross-Validation:", rmse_cross_val)

new_row = {"Model": "Lasso","MAE": mae, "MSE": mse, "RMSE": rmse, "R2 Score": r_squared, "RMSE (Cross-Validation)": rmse_cross_val}
models = models.append(new_row, ignore_index=True)
prediction_all['Lasso']=predictions

MAE: 12.856164091403214
MSE: 263.1413367443284
RMSE: 16.22163175344356
R2 Score: 0.7773923900082731
------------------------------
RMSE Cross-Validation: 15.954995824358576


<ipython-input-16-c88898c1af1d>:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  models = models.append(new_row, ignore_index=True)


Elastic Net

In [17]:
elastic_net = ElasticNet()
elastic_net.fit(X_train, y_train)
predictions = elastic_net.predict(X_test)

mae, mse, rmse, r_squared = evaluation(y_test, predictions)
print("MAE:", mae)
print("MSE:", mse)
print("RMSE:", rmse)
print("R2 Score:", r_squared)
print("-"*30)
rmse_cross_val = rmse_cv(elastic_net)
print("RMSE Cross-Validation:", rmse_cross_val)

new_row = {"Model": "ElasticNet","MAE": mae, "MSE": mse, "RMSE": rmse, "R2 Score": r_squared, "RMSE (Cross-Validation)": rmse_cross_val}
models = models.append(new_row, ignore_index=True)
prediction_all['ElasticNet']=predictions

MAE: 13.138526233805047
MSE: 276.4704237772375
RMSE: 16.627399790022416
R2 Score: 0.7661164869347452
------------------------------
RMSE Cross-Validation: 16.461298319429098


<ipython-input-17-4ed9d696ff0e>:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  models = models.append(new_row, ignore_index=True)


Support Vector Machines

In [ ]:
svr = SVR()
svr.fit(X_train, y_train)
predictions = svr.predict(X_test)

mae, mse, rmse, r_squared = evaluation(y_test, predictions)
print("MAE:", mae)
print("MSE:", mse)
print("RMSE:", rmse)
print("R2 Score:", r_squared)
print("-"*30)
rmse_cross_val = rmse_cv(svr)
print("RMSE Cross-Validation:", rmse_cross_val)

new_row = {"Model": "SVR","MAE": mae, "MSE": mse, "RMSE": rmse, "R2 Score": r_squared, "RMSE (Cross-Validation)": rmse_cross_val}
models = models.append(new_row, ignore_index=True)
prediction_all['SVR']=predictions

MAE: 14.398700400467208
MSE: 395.42230856603555
RMSE: 19.885228401153345
R2 Score: 0.6654876951817659
------------------------------


Random Forest Regressor

In [ ]:
random_forest = RandomForestRegressor(n_estimators=100)
random_forest.fit(X_train, y_train)
predictions = random_forest.predict(X_test)

mae, mse, rmse, r_squared = evaluation(y_test, predictions)
print("MAE:", mae)
print("MSE:", mse)
print("RMSE:", rmse)
print("R2 Score:", r_squared)
print("-"*30)
rmse_cross_val = rmse_cv(random_forest)
print("RMSE Cross-Validation:", rmse_cross_val)

new_row = {"Model": "RandomForestRegressor","MAE": mae, "MSE": mse, "RMSE": rmse, "R2 Score": r_squared, "RMSE (Cross-Validation)": rmse_cross_val}
models = models.append(new_row, ignore_index=True)
prediction_all['RandomForestRegressor']=predictions

XGBoost Regressor

In [ ]:
xgb = XGBRegressor(n_estimators=1000, learning_rate=0.01)
xgb.fit(X_train, y_train)
predictions = xgb.predict(X_test)

mae, mse, rmse, r_squared = evaluation(y_test, predictions)
print("MAE:", mae)
print("MSE:", mse)
print("RMSE:", rmse)
print("R2 Score:", r_squared)
print("-"*30)
rmse_cross_val = rmse_cv(xgb)
print("RMSE Cross-Validation:", rmse_cross_val)

new_row = {"Model": "XGBRegressor","MAE": mae, "MSE": mse, "RMSE": rmse, "R2 Score": r_squared, "RMSE (Cross-Validation)": rmse_cross_val}
models = models.append(new_row, ignore_index=True)
prediction_all['XGBRegressor']=predictions

Polynomial Regression (Degree=2)


In [ ]:
poly_reg = PolynomialFeatures(degree=2)
X_train_2d = poly_reg.fit_transform(X_train)
X_test_2d = poly_reg.transform(X_test)

lin_reg = LinearRegression()
lin_reg.fit(X_train_2d, y_train)
predictions = lin_reg.predict(X_test_2d)

mae, mse, rmse, r_squared = evaluation(y_test, predictions)
print("MAE:", mae)
print("MSE:", mse)
print("RMSE:", rmse)
print("R2 Score:", r_squared)
print("-"*30)
rmse_cross_val = rmse_cv(lin_reg)
print("RMSE Cross-Validation:", rmse_cross_val)

new_row = {"Model": "Polynomial Regression (degree=2)","MAE": mae, "MSE": mse, "RMSE": rmse, "R2 Score": r_squared, "RMSE (Cross-Validation)": rmse_cross_val}
models = models.append(new_row, ignore_index=True)
prediction_all['Polynomial Regression (degree=2)']=predictions

Model Comparison


The less the Root Mean Squared Error (RMSE), The better the model is.

In [ ]:
models.sort_values(by="RMSE (Cross-Validation)")

In [ ]:
models['R2ScorePercentage'] = models['R2 Score'] * 100

In [ ]:
models

In [ ]:
plt.figure(figsize=(12,8))
sns.barplot(x=models["Model"], y=models["RMSE (Cross-Validation)"])
plt.title("Models' RMSE Scores (Cross-Validated)", size=15)
plt.xticks(rotation=30, size=12)
plt.show()

In [ ]:

pd.set_option('display.max_rows', None)
prediction_all[prediction_all['RealRuns']>220]

In [ ]:
prediction_all

* ENSEMBLE
* Combination of several machine learning algorithms
* Different merging methodologies:
* a) Problem
    i) Supervised -> Mean for each row
    ii) Unsupervised -> Max Voting (4)
* b)

CAN Network -> 90%